In [7]:
import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1]  # notebooks → repo root
SRC = ROOT / "src"

sys.path.insert(0, str(SRC))

print("Added:", SRC)


NameError: name '__file__' is not defined

In [5]:
import sys, os
from pathlib import Path
print("cwd:", Path().resolve())
print("sys.path[0:5]:", sys.path[:5])

cwd: /workspaces/lit-nonfiction-weave/notebooks
sys.path[0:5]: ['/workspaces/src', '/usr/local/python/3.12.1/lib/python312.zip', '/usr/local/python/3.12.1/lib/python3.12', '/usr/local/python/3.12.1/lib/python3.12/lib-dynload', '']


In [6]:
from storygraph.router import Pipeline

pipe = Pipeline(seed=137)
state = pipe.run_minimal(
    premise="A day in Kyiv during water outages",
    venue="The Atlantic"
)

state.metrics, list(state.drafts.keys())[:3]


ModuleNotFoundError: No module named 'storygraph'

In [31]:
import importlib
import storygraph.llm
importlib.reload(storygraph.llm)


<module 'storygraph.llm' from '/workspaces/lit-nonfiction-weave/src/storygraph/llm.py'>

In [ ]:
from storygraph.llm import LLMClient, LLMConfig
cfg = LLMConfig(model="anthropic/claude-opus-4-20250514", seed=137)
client = LLMClient(cfg)
print(client.backend[0])


anthropic


Initialize empty state

In [2]:
# Add repo root and src/ to Python path for notebooks
import sys
from pathlib import Path

# notebook running in: repo/src/notebooks
# repo root = parents[2]
ROOT = Path().resolve().parents[2]    
SRC = ROOT / "src"

# attach to sys.path
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("Repo root:", ROOT)
print("SRC added:", SRC)


Repo root: /
SRC added: /src


In [3]:
from storygraph.state import StoryState
from storygraph.router import Pipeline

state = StoryState(
    premise="A day in Kyiv during water outages",
    venue="The Atlantic",
    seed=137
)

state


StoryState(seed=137, premise='A day in Kyiv during water outages', venue='The Atlantic', outline=None, drafts={}, draft_v1_concat='', draft_v2_concat='', claim_graph={}, metrics={}, word_target_low=3000, word_target_high=7500)

In [27]:
import storygraph.llm
print(storygraph.llm.__file__)


/workspaces/lit-nonfiction-weave/src/storygraph/llm.py


In [24]:
import importlib
import storygraph.llm
importlib.reload(storygraph.llm)


<module 'storygraph.llm' from '/workspaces/lit-nonfiction-weave/src/storygraph/llm.py'>

Run Planner

In [ ]:
from storygraph.llm import LLMClient, LLMConfig
cfg = LLMConfig(model="anthropic/claude-opus-4-20250514", seed=137)
client = LLMClient(cfg)
print("Backend:", client.backend[0])


Backend: anthropic


In [22]:
import inspect
import storygraph.llm
print(inspect.getsource(storygraph.llm.LLMClient._select_backend))


    def _select_backend(self, model: str) -> Tuple[str, Any]:
        if model.startswith("openai/") and OpenAI and os.getenv("OPENAI_API_KEY"):
            return ("openai", OpenAI(api_key=os.getenv("OPENAI_API_KEY")))
        if (
            model.startswith("anthropic/")
            and anthropic
            and os.getenv("ANTHROPIC_API_KEY")
        ):
            return (
                "anthropic",
                anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY")),
            )
        return ("local", LocalMockLLM(self.cfg.seed))



Run DraftAgent (all beats)

In [ ]:
state = pipe.run_draft(state)
len(state.drafts), list(state.drafts.keys())


Run FactAgent (simple version)

In [ ]:
state = pipe.run_fact(state)
state.claim_graph


Run RevisionAgent

In [ ]:
state = pipe.run_revision(state)
state.revisions


Save snapshot to disk

In [ ]:
with open("notebooks/output_state.json", "w", encoding="utf8") as f:
    json.dump(state.model_dump(mode="json"), f, indent=2, ensure_ascii=False)

"saved"
